<a href="https://colab.research.google.com/github/Shona173/codes/blob/main/3D_SDF3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install POT
!pip install trimesh
!pip install pyrender
!pip install ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 711.2/711.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 984.0/984.0 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.6 MB/s eta 0:00:00
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1745193 sha256=3aa481c8c22a024ac82d0999781433a6416f0c3d2483559befb89d35d87a7d7f
  Stored in directory: /root/.cache/pip/wheels/2f/37/f5/f88cd3dddf75bc3ce608e44bf8a79078c408bf1f351a50818e
Successfully built PyOpenGL
  Attempting uninstall: PyOpenGL
    Found existing installation: PyOpenGL 3.1.9
    Uninstalling PyOpenGL-3.1.9:
      Successfully uninstalled PyOpenGL-3.1.9


In [2]:
import numpy as np
import numpy.random as random

import matplotlib.pyplot as plt
import skimage.measure
import trimesh
import pyrender
import matplotlib.animation as animation
from google.colab import files
from matplotlib import animation
from scipy.ndimage import gaussian_filter
from IPython.display import HTML
import ot
import os
import time

In [3]:
def clamp(value, min_val, max_val):
    return np.maximum(min_val, np.minimum(value, max_val))

In [4]:
def sdf_to_distribution(f_sdf, grid, grid_size):
    vals = f_sdf(grid)
    mask = (vals < 0).astype(np.float32)
    normed = mask / np.sum(mask)
    return normed.reshape((grid_size, grid_size, grid_size))

In [5]:
def sdf_sphere(p, s=0.5):
    p = p.copy()
    return np.sqrt(np.sum(p**2, axis=1))-s

In [6]:
def sdf_box(p, b=[0.5,0.5,0.5]):
    p2 = p.copy()
    q = np.abs(p2) - b
    q2 = np.maximum(q, 0.0)
    q2 = np.linalg.norm(q2, axis=1, ord=2)
    q3 = np.minimum(np.maximum(q[:,0], np.maximum(q[:,1], q[:,2])), 0.0)
    return q2 + q3

In [7]:
def sdf_octahedron(p, s=0.5):
    p = np.abs(p)
    m = np.sum(p, axis=1) - s
    d = np.zeros(p.shape[0])

    # Get index for each branch
    idx_x = 3.0 * p[:, 0] < m
    idx_y = (~idx_x) & (3.0 * p[:, 1] < m)
    idx_z = (~idx_x) & (~idx_y) & (3.0 * p[:, 2] < m)
    idx_else = ~(idx_x | idx_y | idx_z)

    # Build q for each condition
    q = np.zeros_like(p)
    q[idx_x] = p[idx_x]
    q[idx_y] = p[idx_y][:, [1, 2, 0]]
    q[idx_z] = p[idx_z][:, [2, 0, 1]]

    # Calculate distance for q
    k = clamp(0.5 * (q[:, 2] - q[:, 1] + s), 0.0, s)
    d_tmp = np.linalg.norm(np.stack([q[:, 0], q[:, 1] - s + k, q[:, 2] - k], axis=1), axis=1)

    # Assign to d according to condition
    d[idx_x | idx_y | idx_z] = d_tmp[idx_x | idx_y | idx_z]
    d[idx_else] = m[idx_else] * 0.57735027  # 1/sqrt(3)

    return d

In [8]:
def r_union(f1, f2):
    return f1 + f2 + np.sqrt(f1**2 + f2**2)

In [9]:
def r_intersection(f1,f2):
  return f1+f2-np.sqrt(f1**2+f2**2)

In [10]:
def entropy(mu, a):
    mu_safe = np.maximum(mu, 1e-8)
    return -np.sum(a * mu_safe * np.log(mu_safe))

In [11]:
def convolution_operator(volume, sigma):
    return gaussian_filter(volume, sigma=sigma, mode='constant')

In [12]:
def algorithm_barycenter_3d(mu_list, alpha_list, sigma=0.5, max_iter=100):
    k = len(mu_list)
    shape = mu_list[0].shape
    a = 1.0 / np.prod(shape)

    mu_list = [mu / np.sum(mu) for mu in mu_list]
    v_list = [np.ones(shape) for _ in range(k)]
    w_list = [np.ones(shape) for _ in range(k)]

    for _ in range(max_iter):
        d_list = []
        mu= np.ones(shape)
        for i in range(k):
            conv_v = convolution_operator(v_list[i], sigma)
            w_list[i] = mu_list[i] / (conv_v + 1e-8)
            d_i = v_list[i] * convolution_operator(w_list[i], sigma)
            d_list.append(d_i)
            mu=mu*d_list[i]**alpha_list[i]


        for i in range(k):
            v_list[i] = v_list[i] * mu / (d_list[i] + 1e-8)

    return mu

In [13]:
def export_volume_to_obj(volume,level, bounds, output_file):
    min_val = np.min(volume)
    max_val = np.max(volume)

    if max_val-min_val < 1e-6:
        print("Volume fluctuations are too small")
        return

    level = min_val + 0.5 * (max_val - min_val)

    try:
      verts, faces, normals, _ = skimage.measure.marching_cubes(volume, level=level)
    except RuntimeError as e:
        print("marching_cubes failed: {e}")
        return

    scale = (bounds[1] - bounds[0]) / (volume.shape[0] - 1)
    verts = verts * scale + bounds[0]

    mesh = trimesh.Trimesh(vertices=verts, faces=faces, vertex_normals=normals)
    mesh.export(output_file)

In [14]:
def conv_ot_barycenter(mu_list, alpha, K, niter=100, eps=1e-8):

    mu_list = [mu / np.sum(mu) for mu in mu_list]
    alpha = np.array(alpha, dtype=np.float32)
    alpha = alpha / np.sum(alpha)

    shape = mu_list[0].shape
    v_list = [np.ones(shape, dtype=np.float32) for _ in mu_list]
    bary = np.ones(shape, dtype=np.float32)

    for _ in range(niter):
        w_list = []
        d_list = []

        for i in range(len(mu_list)):
            KT_v = K(v_list[i])
            w = mu_list[i] / (KT_v + eps)
            d = v_list[i] * K(w)
            d = np.maximum(d, eps)
            w_list.append(w)
            d_list.append(d)

        log_d = np.stack([alpha[i] * np.log(d_list[i]) for i in range(len(alpha))], axis=0)
        bary = np.exp(np.sum(log_d, axis=0))

        for i in range(len(mu_list)):
            v_list[i] = v_list[i] * bary / (d_list[i] + eps)

    return bary

In [15]:
def optimal_transport_sdf_blend(
    sdf1, sdf2, grid_size=64, bounds=(-1.5, 1.5),
    ts=[0.0],
    sigma=1.0,
    out_dir="ot_outputs"
):
    os.makedirs(out_dir, exist_ok=True)


    x = np.linspace(bounds[0], bounds[1], grid_size)
    X, Y, Z = np.meshgrid(x, x, x, indexing="ij")
    grid = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)

    mu1 = sdf_to_distribution(lambda p: -sdf1(p), grid, grid_size)
    mu2 = sdf_to_distribution(lambda p: -sdf2(p), grid, grid_size)

    for t in ts:
        alpha_list = [1 - t, t]
        mu_bary = conv_ot_barycenter([mu1, mu2], alpha_list, lambda f: gaussian_filter(f, sigma=sigma), niter=100)#test
        #mu_bary = algorithm_barycenter_3d([mu1, mu2], alpha_list, sigma=sigma, max_iter=100)
        min_mu=np.min(mu_bary)
        max_mu=np.max(mu_bary)
        level = min_mu + 0.5 * (max_mu - min_mu)
        filename = f"t{t:.2f}.obj"
        export_volume_to_obj(mu_bary, level=level, bounds=bounds, output_file=os.path.join(out_dir, filename))
        print("mu1 sum:", np.sum(mu1))
        print("mu2 sum:", np.sum(mu2))
        print("mu_bary sum:", np.sum(mu_bary))
        vals = -sdf_sphere(grid)
        print("sdf min/max:", np.min(vals), np.max(vals))
        print("inside count:", np.sum(vals < 0))

In [16]:
optimal_transport_sdf_blend(
    sdf1 = sdf_sphere,
    sdf2 = sdf_octahedron,
    ts = [1.0],
    grid_size = 64,
    bounds = (-2.0, 2.0),
    sigma = 3.0,
    out_dir = "ot_sigma0.1_sphere_octahedron"
)

mu1 sum: 0.99999994
mu2 sum: 0.9999997
mu_bary sum: 1.0000007
sdf min/max: -2.9641016151377544 0.4450142600771787
inside count: 260136
